## Task 1: Distinguish Navigation vs. Signal Axes

In your notebook, define the "Navigation Axes" (where the measurement is made, e.g., x, y scan positions).

Define the "Signal Axes" (what is measured at each point, e.g., an EELS spectrum or a 2D diffraction pattern).

Use signal.axes manager to print and verify the dimensionality of a 4D-STEM dataset (expected: 2 Navigation, 2 Signal).

In [ ]:
# Your code here
import py4DSTEM
import numpy as np

# For demonstration, we'll create a synthetic 4D-STEM dataset
# In practice, you would load real data with py4DSTEM.io.read()

# Create synthetic data: 10x10 scan positions, 128x128 diffraction pattern
nav_shape = (10, 10)
sig_shape = (128, 128)
data = np.random.rand(*nav_shape, *sig_shape)

# Create a DataCube object
dataset = py4DSTEM.DataCube(data)

# Print axes information
print("Navigation Axes:")
for i, axis in enumerate(dataset.axes_manager.navigation_axes):
    print(f"  Axis {i}: {axis}")

print("\nSignal Axes:")
for i, axis in enumerate(dataset.axes_manager.signal_axes):
    print(f"  Axis {i}: {axis}")

print(f"\nTotal dimensionality: {dataset.axes_manager.ndim}")
print(f"Navigation dimensionality: {len(dataset.axes_manager.navigation_axes)}")
print(f"Signal dimensionality: {len(dataset.axes_manager.signal_axes)}")

## Task 2: Load and Calibrate 4D-STEM Data

Use py4DSTEM.io.read to load a 4D-STEM dataset (e.g., .dm4 or .h5).

Set the scan step size (real space calibration) using dataset.set_scan_step_size().

Perform Center of Mass (CoM) correction using dataset.get_diffraction_shifts() to center the unscattered beam.

In [ ]:
# Your code here
# Note: This code assumes you have a 4D-STEM dataset file.
# For this example, we'll continue with the synthetic dataset.

# In practice, load real data like this:
# dataset = py4DSTEM.io.read('path/to/your/dataset.dm4')

# Set scan step size (real space calibration)
# Assuming 1 nm step size
step_size = 1.0  # in nm
dataset.set_scan_step_size(step_size)
print(f"Scan step size set to: {step_size} nm")

# Perform Center of Mass (CoM) correction
# This centers the unscattered beam in the diffraction patterns
shifts = dataset.get_diffraction_shifts()
dataset.shift_diffraction_patterns(shifts)
print("Center of Mass correction applied.")

# Verify the calibration
print(f"Navigation axes calibration:")
for axis in dataset.axes_manager.navigation_axes:
    print(f"  {axis.name}: scale={axis.scale}, units={axis.units}")

## Task 3: Virtual Detector Reconstruction

Generate a Virtual Bright Field (BF) image by integrating the central transmitted disk.

Generate an Annular Dark Field (ADF) image by integrating the scattered electrons in an outer ring.

Compare the Z-contrast in the ADF image to the diffraction contrast in the BF image.

In [ ]:
# Your code here
import matplotlib.pyplot as plt

# Define virtual detectors
# Bright Field (BF): central disk
center = (64, 64)  # center of diffraction pattern
bf_radius = 10  # radius in pixels

# Annular Dark Field (ADF): outer ring
adf_inner_radius = 20
adf_outer_radius = 50

# Create virtual detectors
bf_detector = py4DSTEM.process.virtual_detector.BF_Detector((center, bf_radius))
adf_detector = py4DSTEM.process.virtual_detector.ADF_Detector((center, adf_inner_radius, adf_outer_radius))

# Generate virtual images
bf_image = dataset.get_virtual_image(bf_detector)
adf_image = dataset.get_virtual_image(adf_detector)

# Plot the images
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

ax1.imshow(bf_image, cmap='gray')
ax1.set_title('Virtual Bright Field (BF) Image')
ax1.set_xlabel('Scan Position X')
ax1.set_ylabel('Scan Position Y')

ax2.imshow(adf_image, cmap='magma')
ax2.set_title('Virtual Annular Dark Field (ADF) Image')
ax2.set_xlabel('Scan Position X')
ax2.set_ylabel('Scan Position Y')

plt.tight_layout()
plt.show()

# Compare contrasts
print(f"BF image range: {bf_image.min():.3f} to {bf_image.max():.3f}")
print(f"ADF image range: {adf_image.min():.3f} to {adf_image.max():.3f}")
print("\nThe BF image shows diffraction contrast (related to crystal orientation),")
print("while the ADF image shows Z-contrast (related to atomic number and thickness).")

## Task 4: Basic 4D-STEM Visualization

Launch the interactive 4D-STEM browser using dataset.show() (if using a local GUI) or py4D.show_image().

Export a publication-quality figure of a virtual ADF image with a scale bar and a perceptually uniform colormap (e.g., magma).

In [ ]:
# Your code here
# Note: Interactive browser requires GUI, may not work in all environments
# dataset.show()  # Uncomment to launch interactive browser

# Export publication-quality figure
fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(adf_image, cmap='magma', origin='lower')
ax.set_title('Virtual ADF Image', fontsize=16)
ax.set_xlabel('Scan Position X (pixels)', fontsize=12)
ax.set_ylabel('Scan Position Y (pixels)', fontsize=12)

# Add scale bar
# Assuming step_size is in nm, and we want a 10 nm scale bar
scale_bar_length = 10  # nm
pixels_per_nm = 1 / step_size
scale_bar_pixels = int(scale_bar_length * pixels_per_nm)

# Add scale bar to plot
from matplotlib.patches import Rectangle
scale_bar = Rectangle((5, 5), scale_bar_pixels, 2, fill=True, color='white')
ax.add_patch(scale_bar)
ax.text(5 + scale_bar_pixels/2, 10, f'{scale_bar_length} nm', ha='center', va='bottom', color='white', fontsize=10)

# Add colorbar
cbar = plt.colorbar(im, ax=ax, shrink=0.8)
cbar.set_label('Intensity (a.u.)', fontsize=12)

plt.tight_layout()
plt.savefig('virtual_adf_figure.png', dpi=300, bbox_inches='tight')
plt.show()

print("Publication-quality figure saved as 'virtual_adf_figure.png'")

## Task 5: Finalize and Submit

Update your README.md with a brief explanation of how virtual detectors allow post-acquisition imaging.

Push the completed Week 02 notebook to your GitHub repository.

Submit the repository link on Canvas.

In [ ]:
# Your code here
# This task involves updating README and pushing to GitHub
# These are manual steps, but here's a summary:

readme_update = """
## Week 02: Multidimensional Data Structures and 4D-STEM Foundations

Virtual detectors in 4D-STEM allow post-acquisition imaging by integrating different regions of the diffraction patterns collected at each scan position. This enables researchers to create various types of images (BF, ADF, etc.) from the same dataset without needing to recollect data, providing flexibility in data analysis and interpretation.
"""

print("README.md update content:")
print(readme_update)

# Git commands (run in terminal):
# git add Week_02/assignments/assignment_02_solutions.ipynb
# git commit -m "Add Week 02 assignment solutions"
# git push origin dev

print("\nGit commands to run:")
print("git add Week_02/assignments/assignment_02_solutions.ipynb")
print("git commit -m \"Add Week 02 assignment solutions\"")
print("git push origin dev")

print("\nSubmit the repository link on Canvas.")